# Filtrado de base de datos companies.json

Importo las librerías.

In [50]:
import pymongo
from pymongo import MongoClient
client = MongoClient("localhost", 27017)
db = client["companies"]

from pandas.io.json import json_normalize

import pandas as pd

In [51]:
data = db.companies.find()
dataset = pd.DataFrame(list(data))
pd.set_option("display.max_columns", 500)
display(dataset.head())

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,deadpooled_month,deadpooled_url,deadpooled_year,description,email_address,external_links,founded_day,founded_month,founded_year,funding_rounds,homepage_url,image,investments,ipo,milestones,name,number_of_employees,offices,overview,partners,permalink,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,NaN,NaN,NaN,2.0,Server Management Software,pr@adventnet.com,[],NaN,NaN,1996.0,[],http://adventnet.com,"{u'available_sizes': [[[150, 55], u'assets/ima...",[],NaN,[],AdventNet,600.0,"[{u'city': u'Pleasanton', u'description': u'He...","<p>AdventNet is now <a href=""/company/zoho-man...",[],abc3,925-924-9500,[],"[{u'is_past': True, u'provider': {u'permalink'...","[{u'is_past': True, u'person': {u'first_name':...","[{u'available_sizes': [[[150, 94], u'assets/im...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
1,52cdef7c4bab8bd675297d92,"{u'acquired_month': 5, u'price_amount': 200000...",[],None,http://www.flektor-blog.com/video_editing_soft...,http://www.flektor-blog.com,games_video,"[{u'competitor': {u'permalink': u'rockyou', u'...",Thu May 31 21:11:51 UTC 2007,http://www.crunchbase.com/company/flektor,NaN,NaN,None,NaN,None,None,[],NaN,NaN,NaN,[],http://www.flektor.com,"{u'available_sizes': [[[150, 68], u'assets/ima...",[],None,[],Flektor,NaN,"[{u'city': u'Culver City', u'description': Non...",<p>Flektor is a rich-media mash-up platform th...,[],flektor,None,"[{u'permalink': u'flektor', u'name': u'Flektor'}]",[],"[{u'is_past': True, u'person': {u'first_name':...",[],"flektor, photo, video",$0,None,Sat Nov 05 08:42:23 UTC 2011,[]
2,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{u'competitor': {u'permalink': u'empressr', u...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,NaN,NaN,NaN,3.0,Online Business Apps Suite,info@zohocorp.com,"[{u'title': u'Zoho Overview', u'external_url':...",15.0,9.0,2005.0,[],http://zoho.com,"{u'available_sizes': [[[150, 55], u'assets/ima...",[],NaN,"[{u'stoned_month': 8, u'stoned_value': None, u...",Zoho,1600.0,"[{u'city': u'Pleasanton', u'description': u'He...","<p>Zoho offers a suite of Business, Collaborat...",[],abc4,1-888-204-3539,"[{u'permalink': u'zoho-office-suite', u'name':...",[],"[{u'is_past': False, u'person': {u'first_name'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{u'embed_code': u'<object width=""430"" height=..."
3,52cdef7c4bab8bd675297d8f,None,[],None,http://feeds.feedburner.com/omnidrive,http://www.omnidrive.com/blog,network_hosting,"[{u'competitor': {u'permalink': u'dropbox', u'...",Sun May 27 03:25:32 UTC 2007,http://www.crunchbase.com/company/omnidrive,15.0,9.0,,2008.0,None,info@omnidrive.com,[],1.0,11.0,2005.0,[{u'investments': [{u'person': {u'first_name':...,http://www.omnidrive.com,"{u'available_sizes': [[[150, 85], u'assets/ima...",[],None,[],Omnidrive,NaN,"[{u'city': u'Palo Alto', u'description': u'', ...","<p>Currently in public beta, Omnidrive makes i...",[],omnidrive,660-675-5052,"[{u'permalink': u'omnidrive', u'name': u'Omnid...",[],"[{u'is_past': True, u'person': {u'first_name':...",[],"storage, sharing, edit, online",$800k,Nomadesk,Tue Jul 02 22:48:04 UTC 2013,[]
4,52cdef7c4bab8bd675297d93,None,"[{u'acquired_month': 5, u'price_amount': 20000...",None,,,web,[],Thu May 31 21:46:57 UTC 2007,http://www.crunchbase.com/company/fox-interact...,NaN,NaN,,NaN,,,[],1.0,6.0,1979.0,[],http://www.newscorp.com,"{u'available_sizes': [[[150, 71], u'assets/ima...","[{u'funding_round': {u'funded_month': 3, u'sou...",None,[],Fox Interactive Media,0.0,"[{u'city': u'Beverly Hills', u'description': u...",<p>Fox Interactive Media (FIM) 

Después de haber visualizado los datos, he decidido que aquellos requisitos que voy a priorizar son:
    1- Encontrarse cerca de empresas que tengan un valor superior a 1 millon de dólares;
    2- Encontrarse cerca de empresas que tengan menos de 10 años de antigüedad (creadas al menos desde 2009)
    3- Encontrarse cerca de lugares que tengan vida nocturna.
    

In [52]:
perfect_place = db.companies.find({"$and":[{"offices.latitude":{"$exists":True}},{"offices.latitude":{"$ne":None}},{"offices.longitude":{"$exists":True}},{"offices.longitude":{"$ne":None}},{"total_money_raised":{"$exists":True}},{"total_money_raised":{"$gte":"$1M"}},{"founded_year":{"$gte":2009}},{"founded_year":{"$ne":None}},{"offices":{"$not":{"$size":0}}}]},{"name":1, "total_money_raised":1, "founded_year":1, "offices":1, "offices.latitude": 1, "offices.longitude": 1, "_id":0})

In [53]:
df = pd.DataFrame(perfect_place)
df.head()

,founded_year,name,offices,total_money_raised
0,2010,GENWI,"[{u'city': u'Los Altos', u'description': u'', ...",$7.1M
1,2013,Fixya,"[{u'city': u'San Mateo', u'description': u'', ...",$8M
2,2010,Needium,"[{u'city': u'Montreal', u'description': u'Need...",$1M
3,2011,RazorGator,"[{u'city': u'Los Angeles', u'description': Non...",$58.8M
4,2012,Skydeck,"[{u'city': u'Palo Alto', u'description': u'', ...",$4M


Ahora voy a limpiar los datos de cada una de las columnas que vamos a utilizar.

In [54]:
def getFirst(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "total_offices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "main_office": principal
    }


first_office = df[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [55]:
df_clean = pd.concat([df,first_office], axis=1)[["name","lat","lng", "main_office","total_offices","total_money_raised","founded_year"]]

In [56]:
df_clean.head()

,name,lat,lng,main_office,total_offices,total_money_raised,founded_year
0,GENWI,33.817100,-111.903500,"{u'type': u'Point', u'coordinates': [-111.9035...",1,$7.1M,2010
1,Fixya,37.566879,-122.323895,"{u'type': u'Point', u'coordinates': [-122.3238...",1,$8M,2013
2,Needium,45.504931,-73.568163,"{u'type': u'Point', u'coordinates': [-73.56816...",1,$1M,2010
3,RazorGator,34.047312,-118.445243,"{u'type': u'Point', u'coordinates': [-118.4452...",1,$58.8M,2011
4,Skydeck,37.564538,-122.325470,"{u'type': u'Point', u'coordinates': [-122.3254...",1,$4M,2012


In [57]:
df_clean = df_clean.fillna(value=0)
df_clean.head()

,name,lat,lng,main_office,total_offices,total_money_raised,founded_year
0,GENWI,33.817100,-111.903500,"{u'type': u'Point', u'coordinates': [-111.9035...",1,$7.1M,2010
1,Fixya,37.566879,-122.323895,"{u'type': u'Point', u'coordinates': [-122.3238...",1,$8M,2013
2,Needium,45.504931,-73.568163,"{u'type': u'Point', u'coordinates': [-73.56816...",1,$1M,2010
3,RazorGator,34.047312,-118.445243,"{u'type': u'Point', u'coordinates': [-118.4452...",1,$58.8M,2011
4,Skydeck,37.564538,-122.325470,"{u'type': u'Point', u'coordinates': [-122.3254...",1,$4M,2012


In [58]:
df_clean.shape

(73, 7)

In [59]:
#crear nueva columna con lista de lng, lat

In [60]:
#df["comb_key"] = df["cust_id"].astype(str) + "----" + df["tran_dt"].astype(str)
df_clean["geo"]=df_clean["lng"].astype(str) + ", " + df_clean["lat"].astype(str)
df_clean

,name,lat,lng,main_office,total_offices,total_money_raised,founded_year,geo
0,GENWI,33.817100,-111.903500,"{u'type': u'Point', u'coordinates': [-111.9035...",1,$7.1M,2010,"-111.9035, 33.8171"
1,Fixya,37.566879,-122.323895,"{u'type': u'Point', u'coordinates': [-122.3238...",1,$8M,2013,"-122.323895, 37.566879"
2,Needium,45.504931,-73.568163,"{u'type': u'Point', u'coordinates': [-73.56816...",1,$1M,2010,"-73.5681635, 45.5049305"
3,RazorGator,34.047312,-118.445243,"{u'type': u'Point', u'coordinates': [-118.4452...",1,$58.8M,2011,"-118.445243, 34.047312"
4,Skydeck,37.564538,-122.325470,"{u'type': u'Point', u'coordinates': [-122.3254...",1,$4M,2012,"-122.3254705, 37.5645375"
5,Handipoints,37.772323,-122.214897,"{u'type': u'Point', u'coordinates': [-122.2148...",1,$800k,2009,"-122.214897, 37.772323"
6,Moblica,32.055400,34.759500,"{u'type': u'Point', u'coordinates': [34.7595, ...",1,$6M,2011,"34.7595, 32.0554"
7,Magento,34.052187,-118.243425,"{u'type': u'Point', u'coordinates': [-118.2434...",1,$22.5M,2010,"-118.243425, 34.052187"
8,Pixsta,51.518190,-0.208979,"{u'type': u'Point', u'coordinates': [-0.208979...",1,£3M,2010,"-0.208979, 51.51819"
9,FirstString,34.052187,-118.243425,"{u'type': u'Point', u'coordinates': [-118.2434...",1,$200k,2011,"-118.243425, 34.052187"


In [61]:
#df['C'] = df.apply(lambda row: row['A'] + row['B'], axis=1)


In [62]:
#df_clean["geo"]=df_clean["lng"]+" "+df_clean["lat"]
#df_clean.head()

In [65]:
pd.DataFrame.to_csv("oficinas.csv")

TypeError: unbound method to_csv() must be called with DataFrame instance as first argument (got str instance instead)

In [45]:
#https://docs.mongodb.com/manual/reference/operator/query/near/index.html
def findNear(geopoint, radio_max_meters=1000):
    return db.first_office.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


# https://developers.google.com/maps/documentation/geocoding/intro
park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}
radio_max_meters = 10000
num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))

SyntaxError: invalid syntax (<ipython-input-45-65b02ab8b41c>, line 21)

In [22]:
!pip3 install python-dotenv --user

In [29]:
from dotenv import load_dotenv
load_dotenv()
import requests
import os

#BASE_URL = "https://api.github.com"
BASE_URL = "https://maps.googleapis.com"
#https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key=YOUR_API_KEY
API_KEY = os.getenv("token_maps")

#with open(".env") as file:
 #   API_KEY= file.read().strip()

name = "Starbucks"

#res = requests.get("maps/api/place/findplacefromtext/json?input={}/%20of%20{}%20{}&inputtype")
res = requests.get("{}/maps/api/place/findplacefromtext/json?inputtype=textquery&fields={}&key={}".format(BASE_URL,name,API_KEY))
#res = requests.get("{}/repos/{}/{}/stargazers".format(BASE_URL,owner, repo))
res.json()

ImportError: No module named dotenv

In [24]:
df_clean.to_json('oficinas.json', orient="records")